# Up & Downsampling

In [1]:
import pandas as pd
import numpy as np

## Dataset

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_feather('removed_duplicated_columns.feather')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726288 entries, 0 to 726287
Columns: 767 entries, HKTLMYY to TLJYWBE
dtypes: float64(742), int64(21), object(4)
memory usage: 4.2+ GB


In [4]:
target = 'TLJYWBE'
df[target] = df[target].apply(lambda v: v >= 1e-5)
df[target].value_counts()

False    726224
True         64
Name: TLJYWBE, dtype: int64

In [5]:
features = [c for c in df.columns if c != target]
training_features, test_features, train_target, test_target = train_test_split(df[features], df[target], test_size=0.25, stratify=df[target])

In [6]:
print(f'Training set shape: {training_features.shape}')
print(f'Training set target shape: {train_target.shape}')
print(f'Test set shape: {test_features.shape}')
print(f'Training set target shape: {test_target.shape}')

Training set shape: (544716, 766)
Training set target shape: (544716,)
Test set shape: (181572, 766)
Training set target shape: (181572,)


In [7]:
test_target.value_counts()

False    181556
True         16
Name: TLJYWBE, dtype: int64

## SMOTE + TomekLink

In [8]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks

In [9]:
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

### Pipeline

In [11]:
dtypes = training_features.dtypes
dtypes.value_counts()

float64    741
int64       21
object       4
dtype: int64

In [12]:
int_features = dtypes[dtypes == 'int64'].index
float_features = dtypes[dtypes == 'float64'].index
cat_features = dtypes[dtypes == 'object'].index
len(int_features), len(float_features), len(cat_features)

(21, 741, 4)

In [13]:
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))

In [14]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('int', numeric_transformer, int_features),
        ('float', numeric_transformer, float_features),
        ('cat', categorical_transformer, cat_features)
    ])

smote = SMOTE(sampling_strategy=0.2, k_neighbors=5)
tomek = TomekLinks(sampling_strategy='majority')

# Create and evaluate the pipeline
resampling_pipeline = ImbPipeline(steps=[('preprocessor', preprocessor),
                              ('smote', smote),
                              ('tomek', tomek)])

In [ ]:
%%time
X_resampled, y_resampled = resampling_pipeline.fit_transform(training_features, train_target)
print("Size of resampled dataset:", X_resampled.shape)
print("Number of positive samples:", y_resampled.sum())
print("Number of negative samples:", len(y_resampled) - y_resampled.sum())

In [52]:
model = RandomForestClassifier(criterion='entropy')
pipeline = ImbPipeline(steps=[('resample', resampling_pipeline),
                              ('classifier', model)])

In [ ]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)